In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys

from pathlib import Path
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

print(os.path.abspath(''))
root_path = os.path.abspath('')+"/../"
print(root_path)
data_folder = root_path+"/data/"
print(os.listdir(data_folder))

from IPython.display import display
import pandas_profiling
from pandas_profiling import ProfileReport
%matplotlib inline




C:\Users\samir\Documents\Python Scripts\kaggle-retail-rocket\notebooks
C:\Users\samir\Documents\Python Scripts\kaggle-retail-rocket\notebooks/../
['.ipynb_checkpoints', 'category_tree.csv', 'cleaned_category.json', 'cleaned_items.json', 'cleaned_items.json.cp', 'cleaned_items_indent.json', 'events.csv', 'events_rating_tupel.json', 'events_rating_tupel_1.json', 'item_properties_part1.csv', 'item_properties_part2.csv']


# Data Understanding


In [2]:
# Loading the files
category_tree = pd.read_csv(data_folder+"category_tree.csv", header= 0)
events = pd.read_csv(data_folder+"events.csv", header= 0)
item_properties_part1 = pd.read_csv(data_folder+"item_properties_part1.csv", header= 0)
item_properties_part2 = pd.read_csv(data_folder+"item_properties_part2.csv", header= 0)
item_properties = pd.concat([item_properties_part1, item_properties_part2])

## Viewing the structure of the files

* Events file
* Categories
* Items

### Events
* Types?
* Distribution (Share of each events)
* Frequent buyers?

### Categories
* Tree structure

### pandas profile of items
* Data Types?
* Number of Features?
* Relevant Features?

In [3]:
# Events file
display(events.head())
# use pandas profiling
#events.profile_report(correlations=None)

#ev_profile.to_html('ev_profile.html')



,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


### learnings
* only 0.8% transactions
* category to rating (weights? view < addtocart < transaction)
* conversion rate?
* what about users with only one "view"? or with no addtocart?
* calculate users and number of each events?

In [4]:
# Categories
#category_tree.profile_report()



category_tree = category_tree.fillna(-1).astype('int')
category_dict = dict()
for index, row in category_tree.iterrows():
    parentid = row['parentid']
    if np.isnan(parentid):
        parentid = None 
    category_dict[row['categoryid']] = {'parent':parentid , 'path':''}

print(category_dict[395])
parents = set(category_tree['parentid'].array)
category = set(category_tree['categoryid'].array)

category_lowest = category

def tree_to_flat(categories, category_id, path=""):
    parent = -1
    if category_id in categories:
        parent = categories[category_id]['parent']
    path = str(int(category_id)) + "/" + path
    
    if (not parent is None) and parent >= 0 :
        path = tree_to_flat(categories, parent, path)        
    return path

#for root_id in root_parents['categoryid']:
#    cat_path = tree_to_flat(category_tree, root_id)


{'parent': -1, 'path': ''}


In [5]:
category_tree.head()


,categoryid,parentid
0,1016,213
1,809,169
2,570,9
3,1691,885
4,536,1691


In [6]:

min_level = 99
max_level = 0
for cat in category_lowest:
    path = tree_to_flat(category_dict, cat, "")
    path = path.strip('/')
    paths = path.split('/')
    min_level = len(paths) < min_level and len(paths) or min_level
    max_level = len(paths) > max_level and len(paths) or max_level
    category_dict[cat]['path'] = path
    
    
    

#print(category_dict[83])


print(min_level, max_level)

#print (category_dict)

1 6


In [7]:
# write categories to json file for easy use
def myconverter(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.datetime):
            return obj.__str__()

import json

cleaned_cat_dict = dict()
#print(category_dict)
for index in category_dict:
    cleaned_cat_dict["_"+str(int(index))] = category_dict[index]


with open(data_folder+'/cleaned_category.json', 'w') as fp:
    json.dump(cleaned_cat_dict, fp, default=myconverter, indent=2)

### learnings/todos for prep
* levels of category?
* 1307 bottom level category
* check on which level items are clustered?
* can we use top 3 levels as category info? cat 761 has 2 levels only

_The file with item properties (item_properties.csv) includes 20 275 902 rows, i.e. different properties, describing 417 053 unique items. File is divided into 2 files due to file size limitations. Since the property of an item can vary in time (e.g., price changes over time), every row in the file has corresponding timestamp. In other words, the file consists of concatenated snapshots for every week in the file with the behaviour data. However, if a property of an item is constant over the observed period, only a single snapshot value will be present in the file._


In [205]:
# Items
#item_properties.profile_report()



In [76]:
item_properties.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


# Data Preparation

In [208]:
## Users
## drop duplicates
print(events.count())
events_reduced = events.drop_duplicates()
print(events_reduced.count())

timestamp        2756101
visitorid        2756101
event            2756101
itemid           2756101
transactionid      22457
dtype: int64
timestamp        2755641
visitorid        2755641
event            2755641
itemid           2755641
transactionid      22457
dtype: int64


In [229]:
## items
import re

#flattening
p = re.compile('^n([-]*\d{1,}[\.\d{0,}]*)$')
flattened_items = dict()
for index, row in item_properties.iterrows():
    item_id = str(row['itemid'])
    ts = row['timestamp']
    value = row['value'] #
    if p.match(value):
        value = float(p.sub("\\1", value))
    
    prop = row['property']
    key_prop = 'property_'+prop
    key_ts = 'property_ts_'+prop
    if not item_id in flattened_items:
        flattened_items[item_id] = dict()
        
    if not key_ts in flattened_items[item_id]:
        flattened_items[item_id][key_prop] = value
        flattened_items[item_id][key_ts] = ts
    else:
        if ts > flattened_items[item_id][key_ts]:
            flattened_items[item_id][key_prop] = value
            flattened_items[item_id][key_ts] = ts
    
            

#print(flattened_items)
    
    
    
    

In [230]:

with open(data_folder+'/cleaned_items_indent.json', 'w') as fp:
    json.dump(flattened_items, fp, default=myconverter, indent=2)

## Users
* relevant Users
* User features?
* Item features?

# Modelling

## Creating the lightfm Datasets

## Training

# Evaluation